In [1]:
import pandas as pd
import numpy as np
import json
import shutil
import os
import ast
import re
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer


# Cargar las stopwords en inglés, francés, alemán, español y ruso
stop_words = set(stopwords.words('english')).union(
    stopwords.words('french'),
    stopwords.words('german'),
    stopwords.words('spanish'),
    stopwords.words('russian')
)


def remove_characters(input_dir, output_dir):
    """
    Recorre todos los archivos CSV en el directorio de entrada, realiza la sustitución en las columnas de cada archivo y guarda los archivos modificados en el directorio de salida. 
    También copia los archivos .json al directorio de salida.

    Parámetros:
        - input_dir: Directorio de entrada
        - output_dir: Directorio de salida
    """
    # Crea el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    # Recorre todos los archivos en el directorio de entrada
    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            # Ruta al archivo de entrada
            input_path = os.path.join(input_dir, filename)
            # Ruta al archivo de salida
            output_path = os.path.join(output_dir, filename)

            # Intenta leer los datos con pandas en formato 'utf-8'
            try:
                df = pd.read_csv(input_path, encoding='utf-8')
            except UnicodeDecodeError:
                # Si falla, intenta con 'ISO-8859-1'
                df = pd.read_csv(input_path, encoding='ISO-8859-1')

            # Realiza la sustitución en todas las columnas
            df = df.replace('\n', ' ', regex=True).replace('\r', ' ', regex=True)

            # Guarda los datos modificados en el archivo de salida
            df.to_csv(output_path, index=False, encoding='utf-8')
        elif filename.endswith('.json'):
            # Ruta al archivo de entrada
            input_path = os.path.join(input_dir, filename)
            # Ruta al archivo de salida
            output_path = os.path.join(output_dir, filename)

            # Copia el archivo .json al directorio de salida
            shutil.copy(input_path, output_path)


def extract_keywords(kw_col):
    # Eliminar todos los caracteres no alfabéticos y convertir a minúsculas
    cleaned_text = re.sub(r'\W+', ' ', kw_col).lower()
    # Dividir el texto en palabras
    words = re.findall(r'\b\w{4,}\b', cleaned_text)
    # Eliminar números de las palabras
    words = [re.sub(r'\d', '', word) for word in words]
    return words


def remove_stopwords(keywords):
    # Eliminar las stopwords
    words = [word for word in keywords if word not in stop_words]
    return words


def load_data(csv_path, json_path, output_path):
    """
    Loads CSV and JSON files, performs preprocessing and returns a DataFrame.

    Parameters:
        - csv_path: Path to the CSV file.
        - json_path: Path to the JSON file.

    Returns:
        - DataFrame with the loaded and preprocessed data.
    """    
    # load CSV data with different encodings
    data = pd.read_csv(csv_path, encoding='utf-8', quotechar='"', quoting=1)
    
    # Asumiendo que 'data' es tu DataFrame
    data = data[data['category_id'] != 29]
    
    # SXXXX
    data['title'].replace('', np.nan, inplace=True)
    data['tags'].replace('', np.nan, inplace=True)
    data = data.dropna(subset=['title', 'tags'])
    
    # transform date
    data['trending_date'] = pd.to_datetime(data['trending_date'], format='%y.%d.%m')
    data['publish_time'] = pd.to_datetime(data['publish_time'])

    # Ahora podemos extraer la hora y el día de la semana
    data['publish_hour'] = data['publish_time'].dt.hour
    data['publish_day_of_week'] = data['publish_time'].dt.dayofweek
    
    # Tratar columnas booleanas
    boolean_col = ['comments_disabled', 'ratings_disabled', 'video_error_or_removed']
    for col in boolean_col:
        data[col] = data[col].astype(int)
    
    # Load JSON data
    with open(json_path, 'r', encoding='utf-8') as f:
            categories = json.load(f)['items']

    # Crear un diccionario para mapear los IDs de las categorías a sus nombres.
    category_dict = {int(cat['id']): cat['snippet']['title'] for cat in categories}

    # Agregar una nueva columna con los nombres de las categorías de los videos
    data['category_name'] = data['category_id'].map(category_dict)
    
    # Create a combined column for video_id and trending_date
    data['id_date'] = data['video_id'] + data['trending_date'].astype(str)
    
    # Filter new data to exclude videos that are already in the old data
    if os.path.exists(output_path):
        old_data = pd.read_csv(output_path)
        old_data['id_date'] = old_data['video_id'] + old_data['trending_date'].astype(str)
        new_data = data[~data['id_date'].isin(old_data['id_date'])]
        data = pd.concat([old_data, new_data])

    # Aplicar las funciones a la columna 'title' y convertir las listas en tuplas
    data['title'] = data['title'].astype(str)
    data['title_kw'] = data['title'].apply(extract_keywords).apply(remove_stopwords)
    data['title_kw'] = data['title_kw'].apply(tuple)

    # Aplicar las funciones a la columna 'tags' y convertir las listas en tuplas
    data['tags'] = data['tags'].astype(str)
    data['tags_kw'] = data['tags'].apply(extract_keywords).apply(remove_stopwords)
    data['tags_kw'] = data['tags_kw'].apply(tuple)

    # Aplicar las funciones a la columna 'description' y convertir las listas en tuplas
    data['description'] = data['description'].astype(str)
    data['description_kw'] = data['description'].apply(extract_keywords).apply(remove_stopwords)
    data['description_kw'] = data['description_kw'].apply(list)
    
    # XXXXX
    data['title_length'] = data['title'].str.len()
    data['description_length'] = data['description'].str.len()
    data['tags_count'] = data['tags_kw'].apply(lambda x: len(x) if isinstance(x, tuple) else 0)
    
    # Logarithms
    data['log_views'] = np.log1p(data['views'])
    data['log_likes'] = np.log1p(data['likes'])
    data['log_dislikes'] = np.log1p(data['dislikes'])
    data['log_comment_count'] = np.log1p(data['comment_count'])
    
    
    # New charatceristics with the abbreviation of the data country
    data['region'] = os.path.basename(csv_path)[:2]
    
    # Create a combined column for video_id and trending_date
    data['id_date'] = data['video_id'] + data['trending_date'].astype(str)
    
    return data




def preprocess_data(input_dir, output_dir):
    """
    1. Load, preprocess and save the data for each region in a separate file. 
    2. Saves all data in a single file.
    
    Parameters:
            - input_dir: Data directory
            - output_dir: Storage directory of the processed data.
    """
    # Create a list to store all data
    all_data = []
    
    # loop over all files in the data directory
    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            # File upload
            csv_path = os.path.join(input_dir, filename)
            json_path = os.path.join(input_dir, filename[:2] + '_category_id.json')
            output_path = os.path.join(output_dir, filename[:2] + '_processed.csv')
            data = load_data(csv_path, json_path, output_path)
    
            # Filter new data to exclude videos that are already in the old data
            if os.path.exists(output_path):
                old_data = pd.read_csv(output_path)
                old_data['id_date'] = old_data['video_id'] + old_data['trending_date'].astype(str)
                new_data = data[~data['id_date'].isin(old_data['id_date'])]
                data = pd.concat([old_data, new_data])
            
            # Add to list
            all_data.append(data)
            
            # Saves the data in a separate file for each region.
            data.to_csv(output_path, index=False)
    
    # Concatenate all data and save in a single file
    all_data = pd.concat(all_data)
    all_data.to_csv(os.path.join(output_dir, 'ALL_data_processed.csv'), index=False)
    
    # Load the 'ALL_data_processed.csv' file
    all_data_processed = pd.read_csv(os.path.join(output_dir, 'ALL_data_processed.csv'))
    
    
    
    # Crear un objeto LabelEncoder
    le = LabelEncoder()
    le.fit(all_data_processed['region'])

    # Transformar los datos en la columna 'region' de all_data_processed
    all_data_processed['region_ncod'] = le.transform(all_data_processed['region'])
    all_data_processed.to_csv(os.path.join(output_dir, 'ALL_data_processed.csv'), index=False)


# Llamamos a la función para eliminar la columna 'comments' de los archivos CSV y copiar los archivos .json
remove_characters('data', 'data/pre_processed')

# We call the function
preprocess_data('data/pre_processed', 'data/processed')

# Eliminar la carpeta 'pre_processed'
folder_path = 'data/pre_processed'
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    os.makedirs(folder_path)
else:
    print("La carpeta no existe")